<a href="https://colab.research.google.com/github/gorovuha/nlp/blob/main/rake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymorphy2

In [ ]:
from __future__ import division
import io
import os
import pymorphy2 as pm
import codecs
import string
import operator
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
m = pm.MorphAnalyzer()

cache = {}

def mparse(w):
  if w not in cache.keys():
    cache[w] = m.parse(w)
  return cache[w]

count = 1

for dirpath, _, filenames in os.walk('/content/drive/MyDrive/in'):
  for filename in filenames:
    words = []
    words2 = ""
    direction = '/content/drive/MyDrive/out/'
    name = 'chuncked_'
    with io.open(os.path.join('/content/drive/MyDrive/in', filename), 'r', encoding='utf-8') as f:
        text = f.read()
        words = text.split()
        print(len(words))
        for i in range(len(words)-1):
            if mparse(words[i])[0].tag.POS == "ADVB" or mparse(words[i])[0].tag.POS == "VERB" or \
                    mparse(words[i][:-1])[0].tag.POS == "VERB" or mparse(words[i])[0].tag.POS == "PRTS" or \
                    mparse(words[i])[0].tag.POS == "INFN" or mparse(words[i])[0].tag.POS == "COMP" or \
                    mparse(words[i])[0].tag.POS == "ADJS" or mparse(words[i])[0].tag.POS == "GRND" or \
                    mparse(words[i])[0].tag.POS == "CONJ":
                words2 += " | " + words[i] + " | "
            elif (mparse(words[i])[0].tag.POS == "NOUN" and mparse(words[i])[0].tag.case == ("gent" or "accs")):
                words2 += words[i] + " | "
            elif mparse(words[i])[0].tag.POS == "NOUN" and mparse(words[i + 1])[0].tag.POS == ("NOUN" or "ADJF") and \
                    mparse(words[i])[0].tag.case != mparse(words[i + 1])[0].tag.case and mparse(words[i + 1])[
                0].tag.case != "gent":
                words2 += words[i] + " | "
            elif (mparse(words[i])[0].tag.POS == "NOUN" and mparse(words[i])[0].tag.case != ("nomn" or "accs") and
                  mparse(words[i + 1])[0].tag.POS == "NOUN" and mparse(words[i + 1])[0].tag.case == (
                          "nomn" or "accs")):
                words2 += words[i] + " | "
            elif (mparse(words[i])[0].tag.POS == None and mparse(words[i][:-1])[0].tag.POS == None and
                  mparse(words[i][1:])[0].tag.POS == None):
                words2 += " | " + words[i] + " | "
            elif (mparse(words[i])[0].tag.POS == "NOUN" and mparse(words[i - 1])[0].tag.POS == "ADJF" or
                  mparse(words[i - 1])[0].tag.POS == "PRTF") and mparse(words[i])[0].tag.case == \
                    mparse(words[i + 1])[0].tag.case:
                words2 += words[i] + " | "
            else:
                words2 += words[i] + " "
            if i%1000==0:
               print(str(i/len(words)*100))
    new_file = io.open(direction + name + str(count) + '.txt', 'w', encoding='utf-8')
    new_file.write(words2)
    new_file.close()
    count += 1
    print(count)

In [ ]:
punkt = open('/content/drive/MyDrive/stopwords/punct.txt', 'r', encoding='utf-8').read()
print(type(punkt))
stop = open('/content/drive/MyDrive/stopwords/stop.txt', 'r', encoding='utf-8').read()
pronouns = open('/content/drive/MyDrive/stopwords/pronouns.txt', 'r', encoding='utf-8').read()
preps = open('/content/drive/MyDrive/stopwords/preps.txt', 'r', encoding='utf-8').read()
mystop = open('/content/drive/MyDrive/stopwords/userstop.txt', 'r', encoding='utf-8').read()

punkt_list = nltk.word_tokenize(punkt)
stop_list = nltk.word_tokenize(stop) + nltk.word_tokenize(pronouns) + nltk.word_tokenize(preps) + nltk.word_tokenize(mystop)


In [ ]:
def isPunct(word):
    return len(word) == 1 and (word in string.punctuation or word in punkt_list)

def isNumeric(word):
    try:
        float(word) if '.' in word else int(word)
        return True
    except ValueError:
        return False

def isInitial(word):
    return len(word) == 2 and word[1] == '.'

In [ ]:
class RakeKeywordExtractor:

    def __init__(self):
        self.stopwords = set(nltk.corpus.stopwords.words())
        self.top_fraction = 1

    def _generate_candidate_keywords(self, sentences):
        phrase_list = []
        for sentence in sentences:
            words = map(lambda x: '|' if x in self.stopwords or x in stop_list or isNumeric(x) or isInitial(x) else x, \
                        nltk.word_tokenize(sentence.lower()))
            phrase = []
            for word in words:
                if word == '|' or isPunct(word):
                    if len(phrase) > 0:
                        phrase_list.append(phrase)
                        phrase = []
                else:
                    phrase.append(word)
        return phrase_list

    def _calculate_word_scores(self, phrase_list):
        word_freq = nltk.FreqDist()
        word_degree = nltk.FreqDist()
        for phrase in phrase_list:
            degree = len(list(filter(lambda x: not isNumeric(x), phrase))) - 1
            for word in phrase:
                word_freq[word] += 1
                word_degree[word] += degree
        word_scores = {}
        for word in word_freq.keys():
            word_scores[word] = word_degree[word] / word_freq[word]
        return word_scores

    def _calculate_phrase_scores(self, phrase_list, word_scores):
        phrase_scores = {}
        for phrase in phrase_list:
            phrase_score = 0
            for word in phrase:
                phrase_score += word_scores[word]
            phrase_scores[' '.join(phrase)] = phrase_score
        return phrase_scores

    def extract(self, text, incl_scores = False):
        sentences = nltk.sent_tokenize(text)
        print('sentences done')
        phrase_list = self._generate_candidate_keywords(sentences)
        print('phrases done')
        word_scores = self._calculate_word_scores(phrase_list)
        print('words done')
        phrase_scores = self._calculate_phrase_scores(phrase_list, word_scores)
        print('scores done')
        sorted_phrase_scores = sorted(phrase_scores.items(), key = operator.itemgetter(1), reverse = True)
        n_phrases = len(sorted_phrase_scores)
        if incl_scores:
            return sorted_phrase_scores[0 : int(n_phrases / self.top_fraction)]
        else:
            return map(lambda x: x[0], sorted_phrase_scores[0 : int(n_phrases / self.top_fraction)])


In [ ]:
rake = RakeKeywordExtractor()
count = 1

for dirpath, _, filenames in os.walk('/content/drive/MyDrive/out'):
  for filename in filenames:
    with io.open(os.path.join('/content/drive/MyDrive/out/', filename), 'r', encoding='utf-8') as f:
        txt = f.read()

    keywords = rake.extract(txt, incl_scores=True)
    direction = '/content/drive/MyDrive/keywords/'
    name = 'keywords_from_'
    new_file = io.open(direction + name + str(count) + '.txt', 'w', encoding='utf-8')
    for eachkwrd in keywords:
        new_file.write(str(eachkwrd[0]) + ";" + str(eachkwrd[1]) + '\n')
    count += 1
    new_file.close()
    print(count)